In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--lang=en")

driver = webdriver.Chrome(options=options)
driver.get('https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc') # Best 250 movies
# driver.get('https://www.imdb.com/search/title/?groups=bottom_250&sort=user_rating,desc')

movie_links = []

# Loop to load all movies by clicking the "50 more" button
while True:
    try:
        # Scroll to the bottom of the page to trigger lazy loading
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Adjust delay if necessary

        # Click the "50 more" button
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "ipc-see-more__button"))
        )
        load_more_button.click()
        time.sleep(5)  # Wait for additional content to load after clicking

    except:
        # Exit loop if "50 more" button is no longer clickable
        break

# Collect movie links from the current page
soup = BeautifulSoup(driver.page_source, 'html.parser')
content_headers = soup.find_all("div", {"class": "ipc-metadata-list-summary-item__c"})

for content in content_headers:
    title_tag = content.find("a", {"class": "ipc-title-link-wrapper"})
    if title_tag:
        movie_url = "https://www.imdb.com" + title_tag['href']
        movie_links.append(movie_url)

# Close the main page and prepare to visit each movie link
driver.quit()

In [3]:
# Look how many movie links we have
print(len(movie_links))

250


In [4]:
# Initialize a new WebDriver instance for movie details extraction
driver = webdriver.Chrome(options=options)
movie_list = []

# Visit each movie page and collect details
for movie_url in movie_links:
    movie_item = []
    
    # Navigate to the movie's detail page
    driver.get(movie_url)
    time.sleep(2)  # Adjust if necessary

    # Parse the movie page and extract movie details
    movie_soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Movie title
    title_tag = movie_soup.find("h1")
    name = title_tag.get_text(strip=True) if title_tag else "N/A"
    print(f'We are in movie: {name}')
    movie_item.append(name)

    # Find the section containing similar movies
    similar_movies_section = movie_soup.find_all("div", {"class": "ipc-shoveler ipc-shoveler--base ipc-shoveler--page0"})
    similar_movies = []

    for similar_movie in similar_movies_section:
        titles = similar_movie.find_all("span", {"data-testid": "title"})
        for title in titles:
            movie_name = title.get_text(strip=True)
            similar_movies.append(movie_name)

    # Convert similar movies list to a string
    similar_movies_str = ", ".join(similar_movies)
    movie_item.append(similar_movies_str)

    # Append movie details to the list
    movie_list.append(movie_item)

# Close the browser
driver.quit()

# Save data to a CSV file
df = pd.DataFrame(movie_list, columns=['Name', 'Similar Movies'])
df.to_csv('top_movie_data_with_similar.csv', index=False)


We are in movie: The Shawshank Redemption
We are in movie: The Godfather
We are in movie: The Dark Knight
We are in movie: The Godfather Part II
We are in movie: Schindler's List
We are in movie: The Lord of the Rings: The Return of the King
We are in movie: 12 Angry Men
We are in movie: The Lord of the Rings: The Fellowship of the Ring
We are in movie: Pulp Fiction
We are in movie: Inception
We are in movie: Fight Club
We are in movie: Forrest Gump
We are in movie: The Lord of the Rings: The Two Towers
We are in movie: The Good, the Bad and the Ugly
We are in movie: 12th Fail
We are in movie: Interstellar
We are in movie: Goodfellas
We are in movie: The Matrix
We are in movie: One Flew Over the Cuckoo's Nest
We are in movie: Star Wars: Episode V - The Empire Strikes Back
We are in movie: Jai Bhim
We are in movie: Se7en
We are in movie: Saving Private Ryan
We are in movie: The Green Mile
We are in movie: The Silence of the Lambs
We are in movie: Spider-Man: Across the Spider-Verse
We a

In [4]:
# Read the CSV file

df = pd.read_csv('C:/Users/nerea/Documents/MasterDTU/SocialGraphs_fall24/Projects/socialGraphs_fall24/FinalProject/csv/bottom_movie_data_with_similar.csv')

# take the Name colum and paste it in a txt file, one row per line
df['Name'].to_csv('bottom_movie_names.txt', index=False, header=False)
